In [1]:
from scipy import *
from numpy import *
import numdifftools as nd
import bqplot.pyplot as plt
from bqplot.interacts import BrushSelector
import ipywidgets as widgets
from IPython.display import display, Latex, HTML, Math
from scipy.misc import derivative
from scipy.special import factorial
from tokenize import TokenError
import sympy as sym
from sympy.parsing.sympy_parser import parse_expr

## Taylor Series Expansions

A Taylor Series Expansion truncated at $n^{th}$ order is a $n^{th}$ order polynomial approximation of a function in the vicinity of the expansion point, $a$, expressed as:

$$f(x) = \sum_{m=0}^n \frac{1}{m!} f^{(m)}(a) (x-a)^m$$

The idea is that the fitting polynomial is the polynomial that has its first n derivatives match the first n derivatives of the true function at the expansion point, $a$.  You can see this by increasing the order of the fitting polynomial with the dropdown below.


## Taylor Series Interactive Explorer

You can use the interactive plotting tool below to follow along with the iterations described above.

Starting with the default function, $e^{2-x}-3$, which is plotted in blue.  The black curve is the Taylor Series approximation of order specified in the order dropdown box.

You can use the mouse to select a new, zoomed area for the plot.  If you zoom badly, or just want to start over, click the **reset limits** button.

You can enter your own function, change the expansion point, and change the domain on which the function is plotted.

In [2]:
def set_function(strFunc):
    try:
        local_func = lambda x: eval(strFunc)
        f = parse_expr(strFunc)
    except (TokenError, ValueError, SyntaxError):
        print('malformed function')
        return None
    return local_func, f

In [8]:
def func(x):
    return exp(-(x-2)) - 3

# initial function
strFunc = "exp(-(x-2)) - 3"
func, fstr = set_function(strFunc)
sym_func = sym.latex(fstr)

# layouts
fig_layout = widgets.Layout(height = "500px", width = "800px")
title_layout = widgets.Layout(display='flex', justify_content='center', width = "70%", 
                              align_items = 'center', border = 'none')
fig_margin_dict = dict(top=30, bottom = 30, left = 20, right = 10)

# title label and formatting
title_format = "<h2><font color='blue'>$f(x) = {}$</font></h2>"
title_label = widgets.HTMLMath(title_format.format(sym_func),
                              layout = title_layout)

# Main figure
main_fig = plt.figure( layout = fig_layout, fig_margin = fig_margin_dict)
nPts = 200
low = -3
high = 3
x = linspace(low, high,nPts)
A = 0

def Taylor(x, n_order = 1, A = 0):
    y = ones(len(x))*func(A)
    for i in range(1, n_order+1):
        fi = nd.Derivative(func, n=i)
        #fi = derivative(func, A, n=i)
        y += fi(A) * (x-A)**i / factorial(i)
        #y += fi * (x-A)**i / factorial(i)
    return y

y = Taylor(x, n_order = 1)
# to control what appears in legends, use "display_legend = True" on items to show and do NOT 
# call plt.legend()

function_curve = plt.plot(x, func(x), 'b-', display_legend = True, labels = ['function'], stroke_width = 3)
point = plt.plot([A], [func(A)], marker = 'circle', colors = ['red'])
expand_curve = plt.plot(x, y, 'k', display_legend = True, labels = ['Taylor Series'])


# function entry
func_label = widgets.Label("Enter function: ")
func_entry = widgets.Text(strFunc, continuous_update = False)

# domain entry
dom_layout = widgets.Layout(width = '75px')
dom_label = widgets.Label("Enter domain: ")
dom_low = widgets.FloatText(value = low, step = 0.1, layout = dom_layout)
dom_high = widgets.FloatText(value = high, step = 0.1, layout = dom_layout)
func_box = widgets.HBox([func_label, func_entry, dom_label, dom_low, dom_high])

# expansion box
init_label = widgets.Label("Enter expansion point: ")
#expansion_point = widgets.FloatText(value = 0.0, step = 0.1, layout = dom_layout)
expansion_point = widgets.FloatSlider(value = A, min = low, max = high, step = 0.01,
                                     readout = True, readout_format = '.2f', continuous_update = True,
                                     style = {'handle_color':'lightblue'})
order_drop = widgets.Dropdown(options = list(range(13)), value = 1, layout = dom_layout)
order_label = widgets.Label("expansion order, n: ")
                              
btn_reset = widgets.Button(description = "reset limits", disabled = True)
iter_box = widgets.HBox([init_label, expansion_point, order_label, order_drop, btn_reset], grid_gap = "10px")


def update_plot(change):
    global func
    strFunc = func_entry.value
    func, fstr = set_function(strFunc)
    title_label.value = title_format.format(sym.latex(fstr))
    low = dom_low.value
    high = dom_high.value
    xlimits.min = low
    xlimits.max = high
    expansion_point.min = low
    expansion_point.max = high
    x = linspace(dom_low.value, dom_high.value, nPts)
    y = func(x)
    # fix y-axis limits:
    ylimits.min = min(y)
    ylimits.max = max(y)
    
    # update main plot
    function_curve.x = x
    function_curve.y = y
    
    # update point
    A = expansion_point.value
    point.x = [A]
    point.y = [func(A)]
    
    # update fit plot
    expand_curve.x = x
    N_expand = order_drop.value
    y_fit = Taylor(x, N_expand, A = A)
    expand_curve.y = y_fit
    
def brush_callback(change):
    brushing = br_sel.brushing
    if not brushing:
        sel = br_sel.selected
        xlimits.min = sel[0][0]
        ylimits.min = sel[0][1]
        xlimits.max = sel[1][0]
        ylimits.max = sel[1][1]
        btn_reset.disabled = False
    br_sel.selected = None

def reset_limits(stuff):
    update_plot("reset")
    
ylimits = plt.ylim(-50, 150)
xlimits = plt.xlim(-3, 3)
plt.hline(0, colors = ['peru'])

br_sel = BrushSelector(x_scale = function_curve.scales['x'], 
                       y_scale = function_curve.scales['y'], marks = [function_curve], color='lightblue')
main_fig.interaction = br_sel
br_sel.observe(brush_callback, names = ['brushing'])

# connections
order_drop.observe(update_plot, names = 'value')
btn_reset.on_click(reset_limits)
expansion_point.observe(update_plot, names = 'value')
func_entry.observe(update_plot, names = 'value')
dom_low.observe(update_plot, names = 'value')
dom_high.observe(update_plot, names = 'value')

widgets.VBox([title_label, main_fig, func_box,  iter_box])


In [ ]:
%%html
<style>
div.input{
    display:none;
}
</style>